#**Tesi Mattia Polticchia**

---



---



In [0]:
!pip install keras==2.3.1
!pip install keras-rl2
!pip install gym-tetris
!pip install nes-py
!pip install tensorflow-gpu==2.0.0-beta1
!pip install numpy==1.16.4

In [0]:
import gym_tetris as gym
print(gym.__path__)
#Check if GPU is used
import tensorflow as tf
tf.test.gpu_device_name()

from google.colab import drive
drive.mount('/content/drive')


In [1]:
import sys
import numpy as no
import pickle

from nes_py.wrappers import JoypadSpace
import gym_tetris
from gym_tetris.actions import MOVEMENT, SIMPLE_MOVEMENT, TRAIN_MOVEMENT

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from tensorflow.keras.optimizers import Adam


from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy, LinearAnnealedPolicy, BoltzmannQPolicy, BoltzmannGumbelQPolicy
from rl.memory import SequentialMemory
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

#v0 reward score
#v1 reward lines
#v2 reward score - penalize height
#v3 reward lines - penalize height

env = gym_tetris.make('TetrisA-v3')
env = JoypadSpace(env, TRAIN_MOVEMENT)


#this can be the output range 0-3 for the move
nb_actions = env.action_space.n
input_shape = env.observation_space.shape

print("Num actions: ", nb_actions)

#Build the NN
model = Sequential()
model.add(Convolution2D(32, (8, 8), strides=(4, 4),input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(64, (4, 4), strides=(2, 2),input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3), strides=(1, 1),input_shape=input_shape))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('softmax'))
print(model.summary())


Num actions:  4
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 59, 63, 32)        6176      
_________________________________________________________________
activation (Activation)      (None, 59, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 30, 64)        32832     
_________________________________________________________________
activation_1 (Activation)    (None, 28, 30, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 28, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 46592)             0         
_________________________________________________________________
dense (Dense)                (None, 512)

In [0]:
policy = BoltzmannGumbelQPolicy()
memory= SequentialMemory(limit=50000, window_length = 1)
dqn = DQNAgent(model = model, nb_actions = nb_actions , memory=memory, train_interval=1,
               nb_steps_warmup=5000, target_model_update=10000, policy=policy)

dqn.compile(Adam(lr=1e-4), metrics=['mae'])

hist = dqn.fit(env, nb_steps=1500000, visualize=False, log_interval=50000)

weights_filename = '/content/drive/My Drive/Colab Notebooks/dqn_{}_weights.h5f'.format("gym_tetris")
dqn.save_weights(weights_filename, overwrite=True)

with open('/content/drive/My Drive/Colab Notebooks/trainHistoryDict', 'wb') as file_pi:
        pickle.dump(hist.history, file_pi)

In [0]:
#@title Reopen
#reopen the fit function but i'm not quite sure for that
weights_filename= '/content/dqn_{}_weights.h5f'.format('gym_tetris')
dqn.load_weights(weights_filename)
dqn.fit(env, nb_steps=500000, visualize=False, log_interval=50000)
dqn.save_weights(weights_filename, overwrite=True)


In [0]:
#@title Reload 
#Reload the weight and make test

policy = BoltzmannQPolicy()
memory= SequentialMemory(limit=50000, window_length = 1)
dqntest = DQNAgent(model = model, nb_actions = num_actions , memory=memory, train_interval=4,
               nb_steps_warmup=5000, target_model_update=100, policy=policy)
dqntest.compile(Adam(lr=1e-4), metrics=['mae'])

weights_filename= '/content/drive/My Drive/Colab Notebooks/dqn_{}_weights.h5f'.format("gym_tetris")
dqntest.load_weights(weights_filename)
nb_episodes = 10 

dqntest.test(env,nb_episodes=nb_episodes,visualize=False)

In [0]:
#@title print model
from tensorflow.keras.utils import plot_model
plot_model(model, show_shapes=True, show_layer_names= True, expand_nested=True,to_file='model.png')